In [ ]:
import pandas as pd 
import numpy as np 

# Alterar caminho do csv aqui!
arquivos = [
    'C:/GitHub/Analise_Eccomerce/Dados/2019-Oct.csv',
    'C:/GitHub/Analise_Eccomerce/Dados/2019-Nov.csv',
    'C:/GitHub/Analise_Eccomerce/Dados/2019-Dec.csv',
    'C:/GitHub/Analise_Eccomerce/Dados/2020-Jan.csv',
    'C:/GitHub/Analise_Eccomerce/Dados/2020-Feb.csv',
]

dfs = []

In [ ]:
# Le todos os arquivos da lista e converte a data pro formato correto 
for caminho in arquivos:
    df_temp = pd.read_csv(caminho)
    df_temp['event_time'] = pd.to_datetime(df_temp['event_time'])
    df_temp['data'] = df_temp['event_time'].dt.strftime('%Y%m%d')  # formato string YYYYMMDD
    dfs.append(df_temp)

# Concatenar tudo em um único DataFrame
df_final = pd.concat(dfs, ignore_index=True)    

In [ ]:
# Criacao coluna mes coletando as 6 primeiras str. EX: 202411
df_final['mes'] = df_final['data'].str[:6]

# Agrupando dados e renomeando coluna de saida.
daily_income = (
    df_final
    .groupby('data', as_index= False)['price']
    .sum()
    .rename(columns={'price': 'daily_total_price'})
)
daily_income = (
    daily_income
    .sort_values('daily_total_price', ascending=False)
    .head(10)
)
monthly_income = (
    df_final
    .groupby('mes', as_index = False)['price']
    .sum()
    .rename(columns={'price': 'monthly_total_price'})
)
sale_category = (
    df_final
    .groupby('category_code', as_index = False)['price']
    .sum()
    .rename(columns={'price': 'category_total_price'})
)

In [ ]:
# Produtos mais vendidos.
event_purchase = df_final[df_final['event_type'] == 'purchase']
most_sold_products = (
    event_purchase['product_id']
    .value_counts()
    .head(10)
    .reset_index()
)
most_sold_products.columns = ['product_id', 'quantity_sold']

# Produtos com maior faturamento.
highest_revenue = (
    event_purchase
    .groupby('product_id', as_index=False)['price']
    .sum()
    .sort_values('price', ascending=False)
    .head(5)
)

In [ ]:
# Conversão: visitas → adição ao carrinho → compra.
funnel_counts =(
    df_final
    .groupby('event_type')['user_id']
    .nunique()
)

view_to_cart = funnel_counts['cart'] / funnel_counts['view']
cart_to_purchase = funnel_counts['purchase'] / funnel_counts['cart']
view_to_purchase = funnel_counts['purchase'] / funnel_counts['view']


In [ ]:
# SAIDAS 

print('=== Dias com mais receita ===')
print(daily_income.to_string(index=False))

print('\n=== Receita em cada mês ===')
print(monthly_income.to_string(index=False))

print('\n=== Maior receita por categoria ===')
print(sale_category.to_string(index=False))




In [ ]:
print('=== Produtos mais vendidos ===')
print(most_sold_products.to_string(index=False))

print('\n=== Produtos com maior faturamento ===')
print(highest_revenue.to_string(index=False))


In [ ]:
print('=== Análise Funil ===\n')

print('Views -> Carrinho:')
print(view_to_cart)

print('\nCarrinho -> Efetivação de Compra:')
print(cart_to_purchase)

print('\nViews -> Efetivação de Compra:')
print(view_to_purchase)
